In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import re
import string
import codecs
import dill as pickle
import pandas as pd
import numpy as np
from collections import Counter

from pythainlp.tokenize import word_tokenize
from pythainlp import word_vector, Tokenizer, sent_tokenize
from pythainlp.util import dict_trie
from pythainlp.ulmfit import *
from pythainlp.corpus.common import thai_words
from pythainlp.corpus import thai_stopwords, wordnet


from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from IPython.display import display

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
stemmer = PorterStemmer()
from nltk.corpus import words
from stop_words import get_stop_words



[nltk_data] Downloading package stopwords to /Users/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# DATA TWITTER - Preprocessing

In [3]:
df = pd.read_excel('/Users/admin/Downloads/0101/alpha/tw_thesis/twitter data test final.xlsx')

In [4]:
fwp = ['FWPthailand','Thanathorn_FWP','SPiyabutr','RangsimanRome']
pt =['PheuThaiParty','sudaratofficiall','ThaksinLive']
pprp =['pprpthailand','prayutofficial']
democrat = ['democratTH','KornGoThailand', 'paritw92']

def condition(Username):
    if Username in fwp:
        return 'อนาคตใหม่'
    if Username in pt:
        return 'เพื่อไทย'
    if Username in pprp:
        return 'พลังประชารัฐ'
    if Username in democrat:
        return 'ประชาธิปัตย์'
    if Username == 'actpartyorg':
        return 'รวมพลังประชาชาติไทย'
    if Username == 'ChartthaiPT':
        return 'ชาติไทยพัฒนา'
    if Username == 'NewEconomicsTH':
        return 'เศรษฐกิจใหม่'
    else:
        return None

df['Party'] = df.Username.apply(condition)

In [5]:
df_act = df[df['Party'] == 'รวมพลังประชาชาติไทย']
df_chartthai = df[df['Party'] == 'ชาติไทยพัฒนา']
df_democrat = df[df['Party'] == 'ประชาธิปัตย์']
df_fwp = df[df['Party'] == 'อนาคตใหม่']
df_neweco = df[df['Party'] == 'เศรษฐกิจใหม่']
df_pt = df[df['Party'] == 'เพื่อไทย']
df_pprp = df[df['Party'] == 'พลังประชารัฐ']

act = df_act[['Text']]
chartthai = df_chartthai[['Text']]
democrat = df_democrat[['Text']]
fwp = df_fwp[['Text']]
neweco = df_neweco[['Text']]
pt = df_pt[['Text']]
pprp = df_pprp[['Text']]
All = [act ,chartthai ,democrat ,fwp ,neweco ,pt ,pprp]
p = ('act' ,'chartthai' ,'democrat' ,'fwp' ,'neweco' ,'pt' ,'pprp')

In [14]:
def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

# def remove_hashtags(tweet):
#     '''Takes a string and removes hashtag information'''
#     tweet = re.sub('(#[ก-๙A-Za-z0-9_]+)','', tweet)
#     return tweet

my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]“”^‘’_`{|}…~•@'

# cleaning master function
def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    #tweet = remove_hashtags(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', '', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing(or single)Choose!
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                            if word not in my_stopwords] # remove stopwords

    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list] # apply word rooter
    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    tweet = ' '.join(tweet_token_list)
    return tweet

def clean_tweet_hashtag(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', '', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                            if word not in my_stopwords] # remove stopwords

    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list] # apply word rooter
    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    post = ' '.join(tweet_token_list)
    return tweet

In [15]:
# apply function in all party
df['tweet'] = df.Text.apply(clean_tweet)
for i in All :
    i['tweet'] = i.Text.apply(clean_tweet)
    # i['tweet_hashtags'] = i.Text.apply(clean_tweet_hashtag)
    i['tweet'].replace('', np.nan, inplace=True)
    i.dropna(inplace=True)

<ipython-input-15-acfadeabcebf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i['tweet'] = i.Text.apply(clean_tweet)
/Users/admin/miniconda3/lib/python3.8/site-packages/pandas/core/series.py:4575: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-15-acfadeabcebf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i.dropna(inplace=True)
<ipython

In [16]:
pt

,Text,tweet
190,ขอยุติการหาเสียงเลือกตั้งทางอิเล็กทรอนิกส์ทุกช...,ขอยุติการหาเสียงเลือกตั้งทางอิเล็กทรอนิกส์ทุกช...
191,ขอขอบพระคุณทุกท่านที่มาร่วมฟังปราศรัยใหญ่ของ #...,ขอขอบพระคุณทุกท่านที่มาร่วมฟังปราศรัยใหญ่ของ #...
193,#Thailand #PheuThai party speakers telling sup...,#thailand #pheuthai parti speaker tell support...
194,#Thailand #PheuThai says only way to win again...,#thailand #pheuthai say way win #dictatorship ...
195,#Thailand #PheuThai linked to #Thaksin ex-#PM ...,#thailand #pheuthai link #thaksin ex#pm readi ...
...,...,...
1374,ผู้นำ...ไม่เพียงแต่ต้องตัดสินใจเฉียบขาดแบบมีวิ...,ผู้นำไม่เพียงแต่ต้องตัดสินใจเฉียบขาดแบบมีวิสัย...
1375,Good Monday EP.6 “คู่แข่งคือคู่ค้า ในโลกยุคใหม...,good monday ep คู่แข่งคือคู่ค้า ในโลกยุคใหม่ ม...
1376,Chin up and keep moving forward! We learn from...,chin keep move forward learn past experi live ...
1377,Good Monday รับมือกับการเปลี่ยนแปลงของโลกกับ ด...,good monday รับมือกับการเปลี่ยนแปลงของโลกกับ ด...


# Model


* tokenizer

In [17]:
th_word=[thai_words()]
th_word=[list(x) for x in th_word][0]

In [18]:
#custom tokenizer
add_words = ['ยุบพรรค',   'ลงพื้นที่', 'ธนาธร','สุดารัตน์ เกยุราพันธุ์',  'ทักษิณ','ปิยบุตร','โรม','ช่อ พรรณิการ์','ชัชชาติ','สุเทพ','ลุงกำนัน',
            'เทพเทือก','ประยุทธ์','ประวิทย์','บิ้กแดง','อภิรักษ์','มิ่งขวัญ','อภิสิทธิ์','พริษฐ์','เสรีพิสุทธ์',
            'เสี่ยหนู','อนุทิน','ไพบูล','กรณ์','มงคลกิตติ์','จุรินทร์','วิษณุ','อุตตม','สมคิด','อนุพงศ์','หญิงหน่อย',
            'สส','สว','รัฐมนตรี','สมาชิกสภาผู้แทนราษฎร','ลากตั้ง',
            'กกต','ปปช','ทักษิณ','คสช','ไม่เพียงแต่',
                                'โซเชี่ยลมีเดีย','อนาคตใหม่','พลังประชารัฐ','เพื่อไทย','ภูมิใจไทย','ประชาธิปัตย์',                       'รวมพลังประชาชาติไทย','ชาติไทยพัฒนา','ชาติไทยพัฒนา','เสรีรวมไทย','กทม','กกต','ประจำ','เศรษฐกิจใหม่']
words = th_word+add_words
words  = list(dict.fromkeys(words)) #drop duplicate
custom_tokenizer = Tokenizer(words,keep_whitespace=False)

In [19]:
# evaluate tokenizer
def tk_t(i):
    print(df['tweet'][i])
    print(custom_tokenizer.word_tokenize(df['tweet'][i]))
    print(word_tokenize(df['tweet'][i],keep_whitespace=False))

In [20]:
tk_t(190)

ขอยุติการหาเสียงเลือกตั้งทางอิเล็กทรอนิกส์ทุกช่องทางตามที่ได้แจ้งต่อประธาน กกต ประจํา กทม ทั้งนี้ เพื่อปฏิบัติตามระเบียบคณะกรรมการการเลือกตั้งว่าด้วยวิธีการหาเสียงและลักษณะต้องห้ามในการหาเสียงเลือกตั้งสมาชิกสภาผู้แทนราษฎร พศ๒๕๖๑ และกฎหมายที่เกี่ยวข้อง #wn #หัวใจคือประชาชน
['ขอ', 'ยุติ', 'การ', 'หาเสียง', 'เลือกตั้ง', 'ทางอิเล็กทรอนิกส์', 'ทุก', 'ช่องทาง', 'ตามที่', 'ได้', 'แจ้ง', 'ต่อ', 'ประธาน', 'กกต', 'ประ', 'จํา', 'กทม', 'ทั้งนี้', 'เพื่อ', 'ปฏิบัติ', 'ตามระเบียบ', 'คณะกรรมการ', 'การเลือกตั้ง', 'ว่าด้วย', 'วิธีการ', 'หาเสียง', 'และ', 'ลักษณะ', 'ต้องห้าม', 'ใน', 'การ', 'หาเสียง', 'เลือกตั้ง', 'สมาชิกสภาผู้แทนราษฎร', 'พศ', '๒๕๖๑', 'และ', 'กฎหมาย', 'ที่', 'เกี่ยวข้อง', '#', 'wn', '#', 'หัวใจ', 'คือ', 'ประชาชน']
['ขอ', 'ยุติ', 'การ', 'หาเสียง', 'เลือกตั้ง', 'ทางอิเล็กทรอนิกส์', 'ทุก', 'ช่องทาง', 'ตามที่', 'ได้', 'แจ้ง', 'ต่อ', 'ประธาน', 'กก', 'ต', 'ประ', 'จํา', 'กทม', 'ทั้งนี้', 'เพื่อ', 'ปฏิบัติ', 'ตามระเบียบ', 'คณะกรรมการ', 'การเลือกตั้ง', 'ว่าด้วย', 'วิธีการ', 'หาเสียง', 'และ', 'ลักษ

In [21]:
df

,Username,Datetime,Retweet count,Favorite count,Text,Party,tweet
0,FWPthailand,2019-03-23 10:08:22,3428,2130,https://t.co/p3aJrAJe5g,อนาคตใหม่,
1,FWPthailand,2019-03-23 09:16:13,2032,2742,เลือก #อนาคตใหม่ อย่าให้ประเทศไทย กลับไปเหมือน...,อนาคตใหม่,เลือก #อนาคตใหม่ อย่าให้ประเทศไทย กลับไปเหมือน...
2,FWPthailand,2019-03-23 09:15:47,844,1491,การปราศรัยใหญ่ที่ทรงพลังที่สุดในประวัติศาสตร์ข...,อนาคตใหม่,การปราศรัยใหญ่ที่ทรงพลังที่สุดในประวัติศาสตร์ข...
3,FWPthailand,2019-03-22 17:09:15,3365,2937,ขอขอบคุณทุกท่าน...อย่าลืม 24 มีนากาอนาคตใหม่ อ...,อนาคตใหม่,ขอขอบคุณทุกท่านอย่าลืม มีนากาอนาคตใหม่ อย่าให...
4,FWPthailand,2019-03-22 15:21:36,8662,4627,"""เราคือคนที่ไม่ยอมอยู่ในสังคมที่ไม่เป็นธรรมแบบ...",อนาคตใหม่,เราคือคนที่ไม่ยอมอยู่ในสังคมที่ไม่เป็นธรรมแบบน...
...,...,...,...,...,...,...,...
1558,RangsimanRome,2019-02-08 03:32:24,0,0,"เปิดชื่อแคนดิเดตนายกฯ ""ทูลกระหม่อมหญิงอุบลรัตน...",อนาคตใหม่,เปิดชื่อแคนดิเดตนายกฯ ทูลกระหม่อมหญิงอุบลรัตนร...
1559,RangsimanRome,2019-02-04 07:24:11,24966,2881,#SaveHakeem ผมเห็นรูปนี้แล้วสะเทือนใจมาก ส่วน...,อนาคตใหม่,#savehakeem ผมเห็นรูปนี้แล้วสะเทือนใจมาก ส่วนห...
1560,RangsimanRome,2019-02-04 04:37:40,169,154,ศาลทหารขอนแก่นจำหน่ายคดีไม่รับร่างรัฐธรรมนูญ แ...,อนาคตใหม่,ศาลทหารขอนแก่นจำหน่ายคดีไม่รับร่างรัฐธรรมนูญ แ...
1561,RangsimanRome,2019-02-04 01:45:02,30,38,ที่นี่ไม่ใช่ กกต. หรือสถานที่รับสมัครเลือกตั้ง...,อนาคตใหม่,ที่นี่ไม่ใช่ กกต หรือสถานที่รับสมัครเลือกตั้ง ...


* LDA

In [22]:
n_samples = 50000
n_features = 5000
n_top_words =  50

In [23]:
tfidf_c = TfidfVectorizer( ngram_range=(3,6),
                    max_df=0.99, min_df=2, max_features=n_features,analyzer='word',
                    stop_words=thai_stopwords(),sublinear_tf=True,
                    tokenizer=custom_tokenizer.word_tokenize)
tfidf_fit_c = tfidf_c.fit_transform(pt['tweet']).toarray()
feature_names_c = tfidf_c.get_feature_names()
index_dict = dict(enumerate(feature_names_c))
lda_c = LatentDirichletAllocation(batch_size=512, doc_topic_prior=None,
            evaluate_every=10, learning_decay=0.7,
            learning_method='online', learning_offset=80.0,
            max_doc_update_iter=100, max_iter=100, mean_change_tol=0.001,
            n_components=9, n_jobs=1, perp_tol=0.1,
            random_state=0, topic_word_prior=None,
            total_samples=n_samples, verbose=0)
lda_c.fit(tfidf_fit_c)
topic_dict = {}
for topic_idx, topic in enumerate(lda_c.components_):
    topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names_c[i])
                    for i in topic.argsort()[:-n_top_words - 1:-1]]
pd.DataFrame(topic_dict)

/Users/admin/miniconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['กระไร', 'กาลนาน', 'ชิ้น', 'ดังที่', 'ดี', 'ดีกว่า', 'ด้อย', 'ตัว', 'ต่อไป', 'ถัดไป', 'ทั่วถึง', 'ทำ', 'ที่จะ', 'ท่าน', 'ท้าย', 'นา', 'บอ', 'บัด', 'ระยะเวลา', 'ล่ะ', 'วันวาน', 'สม', 'สมบูรณ์', 'สํา', 'หน้า', 'หรับ', 'หา', 'อย', 'เกี่ยว', 'เก่า', 'เดี๋ยวนี้', 'เย็น', 'เล่า', 'เสมือน', 'เหมือนกัน', 'แด่', 'แม้น', 'แหล่', 'โง้น', 'โน้น', 'ใด', 'ไว', 'ไหม'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


,Topic 0 words,Topic 1 words,Topic 2 words,Topic 3 words,Topic 4 words,Topic 5 words,Topic 6 words,Topic 7 words,Topic 8 words
0,# เลือกตั้ง #,เลือก เพื่อไทย ถล่มทลาย,# ลุง คืน,ฆ่า สุกร สอน บุตร,กรุงเทพฯ จำนวน ชุด,good monday ep,# หัวใจ ประชาชน,สถาบัน พัฒนา รายได้,ทักษิณ ชินวัตร ep
1,ชัชชาติ สิทธิ พันธุ์,ทุกท่าน รับฟัง การปราศรัย,เรียนเชิญ ทุกท่าน รับฟัง,ฆ่า สุกร สอน,ถ เพชรบุรี ตัด แขวง,​ # พรรคเพื่อไทย,# พรรคเพื่อไทย # หัวใจ,# ศูนย์ สร้าง ผู้ประกอบการ รุ่นใหม่,ดร ทักษิณ ชินวัตร ep
2,เลือกตั้ง # เลือกตั้ง,อนาคต หัวใจ การเลือกตั้ง ประสบความสำเร็จ การเล...,รับฟัง การปราศรัย 📢 พื้นที่ #,สุกร สอน บุตร,ห้วยขวาง กรุงเทพฯ จำนวน ชุด เวลา,พรรคเพื่อไทย จำนวน ชุด,# พรรคเพื่อไทย # หัวใจ ประชาชน,ศูนย์ สร้าง ผู้ประกอบการ,การเปลี่ยนแปลง โลก ดร ทักษิณ
3,ชัชชาติ สิทธิ พันธุ์ เย็น,เพื่อไทย จับมือ ประชาชน,พื้นที่ # พรรคเพื่อไทย,​ ​ #,ตัด แขวง กะปิ เขต ห้วยขวาง กรุงเทพฯ,พรรคเพื่อไทย จำนวน ชุด เวลา,พรรคเพื่อไทย # หัวใจ,ศูนย์ สร้าง ผู้ประกอบการ รุ่นใหม่,รับมือ การเปลี่ยนแปลง โลก ดร
4,สิทธิ พันธุ์ เย็น,เพื่อไทย จับมือ ประชาชน เดินหน้า สู้ ศึก,📢 พื้นที่ # พรรคเพื่อไทย ค,ปราศรัย ครั้งแรก ลาน คนเมือง กทม,ตัด แขวง กะปิ เขต ห้วยขวาง,ผลิต ​ พรรคเพื่อไทย,พรรคเพื่อไทย # หัวใจ ประชาชน,สร้าง ผู้ประกอบการ รุ่นใหม่,การเปลี่ยนแปลง โลก ดร
5,พรรคเพื่อไทย # เลือกตั้ง #,ชัย ผู้สมัคร สส แจ้ง,รับฟัง การปราศรัย 📢,ครั้งแรก ลาน คนเมือง กทม,กะปิ เขต ห้วยขวาง,จำนวน ชุด เวลา,# thailand # pheuthai,# ศูนย์ สร้าง,การเปลี่ยนแปลง โลก ดร ทักษิณ ชินวัตร ep
6,พรรคเพื่อไทย # เลือกตั้ง,อนาคต หัวใจ การเลือกตั้ง,📢 พื้นที่ #,ลาน คนเมือง กทม,ถ เพชรบุรี ตัด,สลาก การออม แห่งชาติ,thailand # pheuthai,# ศูนย์ สร้าง ผู้ประกอบการ,ดร ทักษิณ ชินวัตร
7,ครั้งแรก รอบ ปี,คำตอบ การนำพา ประเทศ สงบสุข,📢 พื้นที่ # พรรคเพื่อไทย,ปราศรัย ครั้งแรก ลาน คนเมือง,กะปิ เขต ห้วยขวาง กรุงเทพฯ จำนวน ชุด,# pm # prayut,# thailand #,ลาน คนเมือง หน้า ศาลาว่าการกรุงเทพมหานคร,โลก ดร ทักษิณ ชินวัตร ep
8,เลือกตั้ง # เลือกตั้ง #,ประสบความสำเร็จ การเลือกตั้ง บริสุทธิ์ ยุติธรร...,รับฟัง การปราศรัย 📢 พื้นที่,ครั้งแรก ลาน คนเมือง,เพชรบุรี ตัด แขวง,pm # prayut,# พรรคเพื่อไทย #,คนเมือง หน้า ศาลาว่าการกรุงเทพมหานคร,การเปลี่ยนแปลง โลก ดร ทักษิณ ชินวัตร
9,# เลือกตั้ง # เลือกตั้ง,ประชาชน เดินหน้า สู้ ศึก,ทุกท่าน รับฟัง การปราศรัย 📢,ปราศรัย ครั้งแรก ลาน,ถ เพชรบุรี ตัด แขวง กะปิ,# thailand # pheuthai,# pm #,ลาน คนเมือง หน้า,รับมือ การเปลี่ยนแปลง โลก ดร ทักษิณ ชินวัตร


In [24]:
X_ = pd.DataFrame(tfidf_fit_c, columns = tfidf_c.get_feature_names())
X_

,# bangkok ralli,# pheuthai parti,# pheuthai parti #,# pheuthai parti # thaielection,# pm #,# pm # prayut,# pm # thaksin,# thaielection #,# thaielection # bangkok,# thaielection # bangkok ralli,...,​ พรรคเพื่อไทย เลขที่ ถ เพชรบุรี,​ พรรคเพื่อไทย เลขที่ ถ เพชรบุรี ตัด,​ ศุภ ชัย,​ ศุภ ชัย เสียใจ,​ ศุภ ชัย เสียใจ เรื่อง,​ ศุภ ชัย เสียใจ เรื่อง เกิดขึ้น,​ ​ #,📢 พื้นที่ #,📢 พื้นที่ # พรรคเพื่อไทย,📢 พื้นที่ # พรรคเพื่อไทย ค
0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.313819,0.0,0.0,0.313819,0.355390,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.405561,0.459284,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.337031,0.381676,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
pt['tweet']

190     ขอยุติการหาเสียงเลือกตั้งทางอิเล็กทรอนิกส์ทุกช...
191     ขอขอบพระคุณทุกท่านที่มาร่วมฟังปราศรัยใหญ่ของ ใ...
192                                                      
193      parti speaker tell support oust chang new gov...
194      say way win go vote say giventhrown away much...
                              ...                        
1374    ผู้นำไม่เพียงแต่ต้องตัดสินใจเฉียบขาดแบบมีวิสัย...
1375    good monday ep คู่แข่งคือคู่ค้า ในโลกยุคใหม่ ม...
1376    chin keep move forward learn past experi live ...
1377    good monday รับมือกับการเปลี่ยนแปลงของโลกกับ ด...
1378    good monday รับมือกับการเปลี่ยนแปลงของโลกกับดร...
Name: tweet, Length: 181, dtype: object

In [38]:
z = tfidf_c.transform([df['tweet'][192]])
l = lda_c.transform(z).flatten().tolist()

In [39]:
l

[0.2, 0.2, 0.2, 0.2, 0.2]

In [10]:
#from above
n_components = 9

In [11]:
def display_topics(data, colname): 
    tfidf = TfidfVectorizer( ngram_range=(1,2),
                        max_df=0.95, min_df=1, max_features=n_features,analyzer='word',
                        stop_words=tuple(thai_stopwords()),sublinear_tf=True,
                        token_pattern='\\b[ก-๙A-Za-z]{6}[ก-๙A-Za-z]+\\b | #[ก-๙A-Za-z0-9_]+')
    tfidf_fit = tfidf.fit_transform(data[colname]).toarray()
    feature_names = tfidf.get_feature_names()
    index_dict = dict(enumerate(feature_names))
    lda = LatentDirichletAllocation(batch_size=512, doc_topic_prior=None,
             evaluate_every=10, learning_decay=0.7,
             learning_method='online', learning_offset=80.0,
             max_doc_update_iter=100, max_iter=100, mean_change_tol=0.001,
             n_components=n_components, n_jobs=1, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=n_samples, verbose=0)
    lda.fit(tfidf_fit)
    topic_dict = {}
    for topic_idx, topic in enumerate(lda.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-n_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [12]:
#without hastags
for i in range(len(p)):
    display('party:{}'.format(p[i]), display_topics(All[i],'tweet'))

'party:act'

,Topic 0 words,Topic 1 words,Topic 2 words,Topic 3 words,Topic 4 words,Topic 5 words
0,อำนาจเจริญแห่สมัครสมาชิก,ปีเปลี่ยนไปเกิดพรรคการเมืองของประชาชนที่แท้จริง,หลักฐานอะไรบ้าง,หนุนพรรคการเมืองของประชาชนเกิดบนเวทีการเมืองไทย,คลิกอ่าน,เก็บตกบรรยากาศ
1,พ่อค้าแม่ขายต้อนรับลุงกำนันและผู้สมัครหน้าใหม่...,ถึงเมืองน้ำดำ,เปิดตัวนักเคลื่อนไหวภาคปชช หวังปักธงจังหวัดเลย,ต้อนรับลุงกำนันและทีมงานคึกคัก หนุนพรรคการเมื...,ด้วยการอุดหนุนภาษีไม่เกิน,เก็บตกบรรยากาศ อำนาจเจริญ
2,กำนันสุเทพ อำนาจเจริญแห่สมัครสมาชิก,ถึงเมืองน้ำดำ ลุงกำนันและว่าที่ผุ้สมัครเดินหา...,ต้องเตรียมเงินคนละกี่บาท หลักฐานอะไรบ้าง,อุทัยธานีแล้ว,ขอเชิญชวนพี่น้องประชาชนร่วมสร้างพรรคการเมืองขอ...,นครสวรรค์แล้ว
3,ยโสธรแล้ว,ชาวเมืองน้ำดำ กาฬสินธุ์ต้อนรับคาราวานคารวะแผ่...,หวังปักธงจังหวัดเลย,ต้อนรับลุงกำนันและทีมงานคึกคัก,ปลุกคนธรรมดาพลิกโฉมการเมืองไทย,นครสวรรค์แล้ว หาสมาขิกและแนะนำพรรคการเมืองของ...
4,กรี๊ดลั่นตลาดรับ,ลุงกำนันและว่าที่ผุ้สมัครเดินหาสมาขิกและแนะนำพ...,เปิดตัวนักเคลื่อนไหวภาคปชช,อุทัยธานีแล้ว ต้อนรับลุงกำนันและทีมงานคึกคัก,ขอเชิญชวนพี่น้องประชาชนร่วมสร้างพรรคการเมืองขอ...,หาสมาขิกและแนะนำพรรคการเมืองของประชาชนที่แท้จริง
5,กำนันสุเทพ,ชาวเมืองน้ำดำ,ต้องเตรียมเงินคนละกี่บาท,สุราษฎร์ธานีจัดประชุมเพื่อตั้งสาขาพรรค เลือกก...,ปลุกคนธรรมดาพลิกโฉมการเมืองไทย คลิกอ่าน,อำนาจเจริญ
6,กรี๊ดลั่นตลาดรับ กำนันสุเทพ,กาฬสินธุ์ต้อนรับคาราวานคารวะแผ่นดินอย่างอบอุ่น,หาสมาชิกและแนะนำพรรคของประชาชน,เลือกกรรมการสาขา,สาขาพรรคเขต มาต้อนรับ,ชาวบ้านพ่อค้าแม่ขาย
7,มหาสารคามแล้ว,สมาชิกรปช โวยสื่อดังบิดเบือน,เพชรบูรณ์แล้ว ถึงเพชรบูรณ์แล้ว,เขตเลือกตั้ง,ร้อยเอ็ดแล้ว,นำว่าที่ผู้สมัคร พรรครปช
8,ชาวบ้านแห่ต้อนรับและให้กำลังใจทีมงาน,หาทิ้งพรรค,ถึงเพชรบูรณ์แล้ว,และทำไพรมารีโหวตด้วย,วันนี้เดินพบชาวร้อยเอ็ดทั้งวัน,พรรครปช
9,เขตคึกคัก,โวยสื่อดังบิดเบือน,ถึงเพชรบูรณ์แล้ว หาสมาชิกและแนะนำพรรคของประชาชน,เข้าเป้า,มาฝั่งอีสาน วันนี้เดินพบชาวร้อยเอ็ดทั้งวัน,พบประชาชน


'party:chartthai'

,Topic 0 words,Topic 1 words,Topic 2 words,Topic 3 words,Topic 4 words,Topic 5 words
0,ศิลปอาชา,รอสรุปผู้สมัคร,อ่านต่อคลิก,ประธานคณะกรรมการนโยบายและยุทธศาสตร์พรรค,เต็มไปด้วยความอบอุ่น จากพี่น้องประชาชน,อัพเดรต
1,คุณกัญจนา,ประธานกรรมการนโยบายและยุทธศาสตร์พรรค,พร้อมสู้ศึกเลือกตั้งพรรคชาติไทยพัฒนา อ่านต่อค...,ศิลปอาชา ประธานคณะกรรมการนโยบายและยุทธศาสตร์พ...,จากพี่น้องประชาชน,หมายเลขผู้สมัครพรรคชาติไทยพัฒนา อัพเดรต
2,คุณกัญจนา ศิลปอาชา,นำทีมพรรคชาติไทยพัฒนา ด้วยบรรยากาศลุย,พร้อมสู้ศึกเลือกตั้งพรรคชาติไทยพัฒนา,ศิลปอาชา,บรรยากาศการปราศรัย,หมายเลขผู้สมัครพรรคชาติไทยพัฒนา
3,ศิลปอาชา หัวหน้าพรรค,ประธานกรรมการนโยบายและยุทธศาสตร์พรรค นำทีมพรร...,คณะกรรมการบริหารพรรคชาติไทยพัฒนา,เลือกตั้ง,จากพี่น้องประชาชน พรรคชาติไทยพัฒนา,คุณกัญจนา
4,คุณวราวุธ,นำทีมพรรคชาติไทยพัฒนา,จังหวัด,จบปัญหา,เต็มไปด้วยความอบอุ่น,ลุยศึกเลือกตั้ง
5,หัวหน้าพรรค,คุณวราวุธ,บึงแก่นนคร จังหวัด,คุณท็อป,พรรคชาติไทยพัฒนา,ลุยศึกเลือกตั้ง เจาะลึกทั่วไทย
6,คุณวราวุธ ศิลปอาชา,ด้วยบรรยากาศลุย,ลงพื้นที่ปราศรัยหาเสียง,หัวข้อดีเบต,ตลาดศรีนคร เต็มไปด้วยความอบอุ่น,ถอดรหัสแท็กติก
7,หัวหน้าพรรค คุณวราวุธ,ศิลปอาชา ประธานกรรมการนโยบายและยุทธศาสตร์พรรค,จังหวัด ขอนแก่น,ศิลปอาชา ในรายการดีเบต,บรรยากาศการปราศรัย ตลาดศรีนคร,ถอดรหัสแท็กติก ลุยศึกเลือกตั้ง
8,พร้อมคณะพรรคชาติไทยพัฒนา,คุณวราวุธ ศิลปอาชา,ลงพื้นที่ปราศรัยหาเสียง บึงแก่นนคร,ในรายการดีเบต เลือกตั้ง,ลงพื้นที่หาเสียง ไหว้ศาลหลักเมือง,เจาะลึกทั่วไทย
9,ศิลปอาชา ประธานนโยบายและยุทธศาสตร์พรรค,พร้อมเพรียงทั้ง คุณกัญจนา,ขอนแก่น ท่ามกลางประชาชนกว่า,หัวข้อดีเบต จบปัญหา,ที่จังหวัดสมุทรสาคร,ด้วยรอยยิ้มสดใส


'party:democrat'

...

,Topic 0 words,Topic 1 words,Topic 2 words,Topic 3 words,Topic 4 words,Topic 5 words
0,มิ่งขวัญ,อย่าได้ลังเลครับ,สวัสดีครับ,เมื่อทุกธุรกิจแยกจากกันจะมีการแข่งขันเพื่อสร้า...,คุณมิ่งขวัญ,คลิปเต็ม
1,นายมิ่งขวัญ,ถ้าเชื่อใจและเห็นด้วยกับเรา,ลุงมิ่ง,คลิปเต็ม,วาระประชาชน กำหนดอนาคตประเทศไทย,standard
2,ขอบคุณนะครับ,อย่าได้ลังเลครับ เศรษฐกิจประเทศไทยต้องทั้งระบบ,หัวหน้า,ทางช่อง,กำหนดอนาคตประเทศไทย,ด้วยระบบเศรษฐกิจใหม่จะทำให้ประเทศไทยเป็นศูนย์ก...
3,ขอบคุณมากครับ,เศรษฐกิจประเทศไทยต้องทั้งระบบ,ควันหลง,หมายเลข,วาระประชาชน,ด้วยระบบเศรษฐกิจใหม่จะทำให้ประเทศไทยเป็นศูนย์ก...
4,standard,ถ้าเชื่อใจและเห็นด้วยกับเรา อย่าได้ลังเลครับ,ลุยถิ่นวัยรุ่น,ล้านล้าน,ชมย้อนหลัง,จะทำได้อย่างไร
5,ล้านล้านบาท,ขอบคุณครับ,สวยพี่สวย,อ่านต่อ,ชมย้อนหลัง วาระประชาชน,เราจะหาเงินได้จากไหน
6,แสงสุวรรณ,ชมสดทาง,หวังเงินเข้าไทย,ตลาดดอกไม้โลก ที่หายไป,รายการพิเศษ,เราจะหาเงินได้จากไหน จะทำได้อย่างไร
7,ข่าวช่องวัน,โค้งสุดท้าย,ลุงมิ่งบินไกลถึงลอนดอนแล้วจ้าา,ที่หายไป คลิปเต็ม,เป็นต้นไป,แล้วเจอกันนะครับ
8,facebook,คลิปเต็ม,โจทย์สำคัญต้องตอบ,ปากคลองตลาด ตลาดดอกไม้โลก,คุณมิ่งขวัญ แล้วพบกันนะครับ,เจอลุงมิ่งยังครับ
9,มิ่งขวัญ แสงสุวรรณ,ยาฆ่าหญ้า,ระบบเศรษฐกิจใหม่คือการตัดระบบผูกขาดและการครอบง...,ตลาดดอกไม้โลก,แล้วพบกันนะครับ,ลุงมิ่งขวัญ


'party:pt'

,Topic 0 words,Topic 1 words,Topic 2 words,Topic 3 words,Topic 4 words,Topic 5 words
0,ลานคนเมือง,พรรคเพื่อไทย,support,หนึ่งในอีกหลายมาตรการแก้วิกฤติเศรษฐกิจของพรรคเ...,ขอบคุณที่แจ้งมา,ชินวัตร
1,chadchart,เขตห้วยขวาง กรุงเทพฯ,atmospher,เลือกตั้ง,เพื่อได้เข้าไปแก้ปัญหาเศรษฐกิจให้พี่น้อง,รับมือการเปลี่ยนแปลงของโลกกับ
2,หน้าศาลาว่าการกรุงเทพมหานคร,เขตห้วยขวาง,speaker,หนึ่งในอีกหลายมาตรการแก้วิกฤติเศรษฐกิจของพรรคเ...,แจ้งผู้เกี่ยวข้องให้แล้วครับ,รับมือการเปลี่ยนแปลงของโลกกับ ชินวัตร
3,ลานคนเมือง หน้าศาลาว่าการกรุงเทพมหานคร,กรุงเทพฯ,ห้วยขวาง,เรียนเชิญทุกท่านร่วมรับฟังการปราศรัย,สามารถตรวจสอบชื่อทางช่องทางนี้ได้ครับ,ตั้งแต่เวลา
4,ลานคนเมืองเริ่มคึกคัก,พรรคเพื่อไทย เขตห้วยขวาง,ด้วยนโยบาย,ในพื้นที่ต่างๆ,ชินวัตร,ตั้งแต่เวลา เป็นต้นไป
5,เพื่อการเติบโตอย่างมีอนาคตและยั่งยืน,ตามวันเวลาที่ปรากฏ,ประชาชน,เรียนเชิญทุกท่านร่วมรับฟังการปราศรัย ในพื้นที...,สร้างความเชื่อมั่นให้เศรษฐกิจ,เป็นต้นไป
6,ลานคนเมือง ศาลาว่าการกรุงเทพมหานคร,กรุงเทพฯ ตามวันเวลาที่ปรากฏ,address,ในกิจกรรม,หัวใจที่ทำให้การเลือกตั้งประสบความสำเร็จอยู่ที...,เพื่อไทย
7,ศาลาว่าการกรุงเทพมหานคร,ผลิตโดย,ได้เวลา ตลาดต้นลำไย,ในกิจกรรม งานนี้เลยต้องขอน้องจินนี่มาเป็นตัวช...,เดินหน้าสู้ศึก,ขอเรียนเชิญทุกท่านที่หัวใจตรงกันมาพบปะกันตามวั...
8,ในคูหาใกล้บ้านท่าน,ผลิตโดย พรรคเพื่อไทย,ได้เวลา,เพื่อชัยชนะที่ถล่มทลายของพรรคฝ่ายประชาธิปไตย,เดินหน้าสู้ศึก มีอนาคต,จะลงพื้นที่พบปะประชาชน
9,ลานคนเมือง ก็ขอให้ทุกท่านเดินทางกลับบ้านโดยสว...,เพื่อไทยปราศรัยใหญ่กทม,และสังคม ประชาชน,เติมกำลังใจในการหาเสียงเลือกตั้งครั้งนี้มาก,หัวใจที่ทำให้การเลือกตั้งประสบความสำเร็จอยู่ที...,เป็นต้นไป ขอเรียนเชิญทุกท่านที่หัวใจตรงกันมาพ...


'party:pprp'

,Topic 0 words,Topic 1 words,Topic 2 words,Topic 3 words,Topic 4 words,Topic 5 words
0,สาวนายน,จันทร์โอชา,พรรคพลังประชารัฐ,เข้าคูหากา,disrupt,ไกรเสริม
1,ผลิตโดย,ค่าแรงขั้นต่ำ,ภูตระกูล,พลังประชารัฐ,ที่ทำการพรรคพลังประชารัฐ ติดตามข่าวสารของพรรค...,เครดิตภาพ
2,หัวหน้าพรรคพลังประชารัฐ,บัตรประชารัฐ,อ่านต่อ,ก้าวข้ามความขัดแย้ง,ที่ทำการพรรคพลังประชารัฐ,ไกรเสริม โตทับเที่ยง
3,สาวนายน หัวหน้าพรรคพลังประชารัฐ,ค่าแรงขั้นต่ำ บาทคืออะไร,จากนี้ไป,เลือกความสงบ,ติดตามข่าวสารของพรรคพลังประชารัฐ,โตทับเที่ยง
4,คนพิการ,บาทคืออะไร,รวยยังไง,bangkok,thailand,เลือกตั้ง
5,ย้อนหลังค่ะ,ปีที่ผ่านมา,พี่น้องชาว,symphoni,มีสัดส่วนหนี้ลงครั้งแรกในรอบ,ผู้สมัคร
6,มีแนวนโยบายและที่มางบประมาณ,ที่ผ่านมา,อนาคตของชาติจากโครงการ,bangkok เข้าคูหากา,รวยอะไรบ้าง,อย่าลังเลใจ
7,สู้ๆนะฮะ,ตั้งแต่เวลา,อาจไม่ไพเราะ,เพศสภาพ,instagram,ยิ่งสงบ
8,ทุกรุ่น,ตั้งแต่เวลา เป็นต้นไป,ปราศรัย,เคารพการตัดสินใจของ,ติดตามข่าวสารของพรรคพลังประชารัฐ instagram,ยิ่งมั่นคง
9,นายอุตตม,เป็นรัฐบาล,โปรไฟล์ผู้สมัครกทม แต่ที่สำคัญคือพร้อมพัฒนาให...,หนังสือคือเพื่อนเราทุกคน,deepest,ยิ่งสงบ ยิ่งมั่นคง
